In [8]:
import pandas as pd
import numpy as np
# from myUtils import pickle_load, symb_perf_stats_vectorized, symb_perf_stats_vectorized_v1
from myUtils import pickle_load, symb_perf_stats_vectorized_v2

file_close = 'df_close_clean'
path_dir = "C:/Users/ping/MyDrive/stocks/yfinance"
path_data_dump = path_dir + "/VSCode_dump/"

df_c = pickle_load(path_data_dump, file_close)
df_c = df_c[-252::]
format1 = 'df_c({}):\n{}\n{}'.format(len(df_c), df_c.head(3), df_c.tail(3))
# print(format1)

In [10]:
def perf_eval(df_close):
  '''
  df_close is a dataframe with date index, columns of symbols' closing price, and symbols as column names.
  The function first calculates symbols' drawdown, Ulcer-Index, max-drawdown, std(returns),
  std(returns)/Ulcer-Index, CAGR, CAGR/std(returns), CAGR/Ulcer-Index. Then it calculates and returns 
  the group's (i.e. all the symbols) mean, standard-deviation, mean/standard-deviation for
  'retrun_std/UI', 'CAGR/return_std', 'CAGR/UI'.

  Args:
    df_close(dataframe): dataframe of symbols' close with
      DatetimeIndex e.g. (['2016-12-19', ... '2016-12-22']), symbols as
      column names, and symbols' close as column values.

  Return:
    df_perf(dataframe): dataframe with columns symbol, first date, last date, Year, CAGR,	UI,
    	return_std/UI, CAGR/return_std, CAGR/UI
    grp_retnStd_d_UI(list): [(std(returns)/Ulcer-Index).mean, (std(returns)/Ulcer-Index).std,
      (std(returns)/Ulcer-Index).mean / (std(returns)/Ulcer-Index).std]
    grp_CAGR_d_retnStd(list): [(CAGR/std(returns)).mean, (CAGR/std(returns)).std,
      (CAGR/std(returns)).mean / (CAGR/std(returns)).std]
    grp_CAGR_d_UI(list): [(CAGR/Ulcer_Index).mean, (CAGR/Ulcer_Index).std,
      (CAGR/Ulcer_Index).mean / (CAGR/Ulcer_Index).std]
  '''

  from myUtils import symb_perf_stats_vectorized_v2

  symbols, period_yr, drawdown, UI, max_drawdown, returns_std, returns_std_div_UI, CAGR, CAGR_div_returns_std, CAGR_div_UI = \
      symb_perf_stats_vectorized_v2(df_close)

  caches_perf_stats_vect = []
  for symbol in symbols:
      date_first = df_close.index[0].strftime('%Y-%m-%d')
      date_last = df_close.index[-1].strftime('%Y-%m-%d')

      cache = (symbol, date_first, date_last, period_yr, CAGR[symbol],
              UI[symbol], returns_std_div_UI[symbol], CAGR_div_returns_std[symbol], CAGR_div_UI[symbol])
      # append performance data (tuple) to caches_perf_stats (list)
      caches_perf_stats_vect.append(cache)

  column_names = ['symbol', 'first date', 'last date', 'Year', 'CAGR',
                  'UI', 'return_std/UI', 'CAGR/return_std', 'CAGR/UI']
  # write symbols' performance stats to dataframe
  df_perf = pd.DataFrame(caches_perf_stats_vect, columns=column_names)

  _cols = ['CAGR', 'UI', 'retrun_std/UI', 'CAGR/return_std', 'CAGR/UI']

  _mean = df_perf['CAGR/UI'].mean()
  _std = df_perf['CAGR/UI'].std()
  grp_CAGR_d_UI = [_mean, _std, _mean/_std]

  _mean = df_perf['CAGR/return_std'].mean()
  _std = df_perf['CAGR/return_std'].std()
  grp_CAGR_d_retnStd = [_mean, _std, _mean/_std]

  _mean = df_perf['return_std/UI'].mean()
  _std = df_perf['return_std/UI'].std()
  grp_retnStd_d_UI = [_mean, _std, _mean/_std]

  return df_perf, grp_retnStd_d_UI, grp_CAGR_d_retnStd, grp_CAGR_d_UI

In [11]:
df_perf, grp_retnStd_d_UI, grp_CAGR_d_retnStd, grp_CAGR_d_UI = perf_eval(df_c)
print(f'grp(retnStd/UI):   mean, std, mean/std: {grp_retnStd_d_UI[0]:>10.6f}, {grp_retnStd_d_UI[1]:>10.6f}, {grp_retnStd_d_UI[2]:>10.6f}')
print(f'grp(CAGR/retnStd): mean, std, mean/std: {grp_CAGR_d_retnStd[0]:>10.6f}, {grp_CAGR_d_retnStd[1]:>10.6f}, {grp_CAGR_d_retnStd[2]:>10.6f}')
print(f'grp(CAGR/UI):      mean, std, mean/std: {grp_CAGR_d_UI[0]:>10.6f}, {grp_CAGR_d_UI[1]:>10.6f}, {grp_CAGR_d_UI[2]:>10.6f}')
df_perf.head()

grp(retnStd/UI):   mean, std, mean/std:   0.140217,   0.067787,   2.068509
grp(CAGR/retnStd): mean, std, mean/std:  -4.050930,  11.079028,  -0.365639
grp(CAGR/UI):      mean, std, mean/std:   0.042430,   2.497449,   0.016989


,symbol,first date,last date,Year,CAGR,UI,return_std/UI,CAGR/return_std,CAGR/UI
0,A,2022-01-06,2023-01-06,1.0,-0.003208,0.130229,0.170473,-0.144511,-0.024635
1,AA,2022-01-06,2023-01-06,1.0,-0.200971,0.417282,0.100086,-4.812067,-0.481620
2,AAL,2022-01-06,2023-01-06,1.0,-0.236403,0.263413,0.135391,-6.628654,-0.897460
3,AAON,2022-01-06,2023-01-06,1.0,-0.021385,0.238489,0.104611,-0.857164,-0.089669
4,AAP,2022-01-06,2023-01-06,1.0,-0.339685,0.230212,0.102019,-14.463286,-1.475528


In [12]:
# %%timeit -n 50 -r 20
my_symbols = ['AE', 'FTSM', 'RMBS', 'WNC']
df_temp = df_c[my_symbols]
df_perf1, grp_retnStd_d_UI, grp_CAGR_d_retnStd, grp_CAGR_d_UI = perf_eval(df_temp)
print(f'grp(retnStd/UI):   mean, std, mean/std: {grp_retnStd_d_UI[0]:>10.6f}, {grp_retnStd_d_UI[1]:>10.6f}, {grp_retnStd_d_UI[2]:>10.6f}')
print(f'grp(CAGR/retnStd): mean, std, mean/std: {grp_CAGR_d_retnStd[0]:>10.6f}, {grp_CAGR_d_retnStd[1]:>10.6f}, {grp_CAGR_d_retnStd[2]:>10.6f}')
print(f'grp(CAGR/UI):      mean, std, mean/std: {grp_CAGR_d_UI[0]:>10.6f}, {grp_CAGR_d_UI[1]:>10.6f}, {grp_CAGR_d_UI[2]:>10.6f}')
df_perf1

grp(retnStd/UI):   mean, std, mean/std:   0.129584,   0.042415,   3.055153
grp(CAGR/retnStd): mean, std, mean/std:   4.735633,  11.572198,   0.409225
grp(CAGR/UI):      mean, std, mean/std:   0.933073,   1.508360,   0.618601


,symbol,first date,last date,Year,CAGR,UI,return_std/UI,CAGR/return_std,CAGR/UI
0,AE,2022-01-06,2023-01-06,1.0,0.353900,0.136485,0.185442,13.982531,2.592952
1,FTSM,2022-01-06,2023-01-06,1.0,-0.005185,0.005453,0.082598,-11.512328,-0.950899
2,RMBS,2022-01-06,2023-01-06,1.0,0.307501,0.199286,0.129011,11.960308,1.543011
3,WNC,2022-01-06,2023-01-06,1.0,0.133588,0.244116,0.121283,4.512022,0.547231


In [13]:
df_SPY = df_c[['SPY']].copy()
df_SPY
df_perf, grp_retnStd_d_UI, grp_CAGR_d_retnStd, grp_CAGR_d_UI = perf_eval(df_SPY)
print(f'grp(retnStd/UI):   mean, std, mean/std: {grp_retnStd_d_UI[0]:>10.6f}, {grp_retnStd_d_UI[1]:>10.6f}, {grp_retnStd_d_UI[2]:>10.6f}')
print(f'grp(CAGR/retnStd): mean, std, mean/std: {grp_CAGR_d_retnStd[0]:>10.6f}, {grp_CAGR_d_retnStd[1]:>10.6f}, {grp_CAGR_d_retnStd[2]:>10.6f}')
print(f'grp(CAGR/UI):      mean, std, mean/std: {grp_CAGR_d_UI[0]:>10.6f}, {grp_CAGR_d_UI[1]:>10.6f}, {grp_CAGR_d_UI[2]:>10.6f}')
df_perf.head()

grp(retnStd/UI):   mean, std, mean/std:   0.104214,        nan,        nan
grp(CAGR/retnStd): mean, std, mean/std: -11.128036,        nan,        nan
grp(CAGR/UI):      mean, std, mean/std:  -1.159696,        nan,        nan


,symbol,first date,last date,Year,CAGR,UI,return_std/UI,CAGR/return_std,CAGR/UI
0,SPY,2022-01-06,2023-01-06,1.0,-0.170663,0.147162,0.104214,-11.128036,-1.159696
